### <center>MLZC Mid-Term project - Q4/2022<img align="right" width="60" height="60" src="../media/heart.png">

# 31Tree

In [1]:
import pandas as pda
import numpy as npy

npy.__version__ , pda.__version__

('1.23.4', '1.5.1')

In [2]:
import pickle
from IPython.display import display

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.__version__ 

'0.12.1'

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_text

from sklearn.ensemble import RandomForestClassifier

In [5]:
import xgboost as xgb

#### Reload data from previous notebooks

In [6]:
# use TRAin FULL dataset
strFil = '.\pickle\dstTraFul.bin'
with open(strFil, 'rb') as filIn: 
    dfrTraFul, y_TraFul = pickle.load(filIn)
dfrTraFul.shape , len(y_TraFul)

((239, 13), 239)

In [7]:
# use TRAin dataset
strFil = '.\pickle\dstTra.bin'
with open(strFil, 'rb') as filIn: 
    dfrTra, y_Tra = pickle.load(filIn)
dfrTra.shape , len(y_Tra)

((179, 12), 179)

In [8]:
# use TST dataset
strFil = '.\pickle\dstTst.bin'
with open(strFil, 'rb') as filIn: 
    dfrTst, y_Tst = pickle.load(filIn)
dfrTst.shape , len(y_Tst)

((60, 12), 60)

In [9]:
# check wil VALidation dataset
strFil = '.\pickle\dstVal.bin'
with open(strFil, 'rb') as filIn: 
    dfrVal, y_Val = pickle.load(filIn)
dfrVal.shape , len(y_Val)

((60, 12), 60)

In [10]:
strFil = '.\pickle\strTar.bin'
with open(strFil, 'rb') as filIn: 
    strTar = pickle.load(filIn)
strTar

'death_event'

In [11]:
strFil = '.\pickle\dfrFea.bin'
with open(strFil, 'rb') as filIn: 
    dfrFea = pickle.load(filIn)
dfrFea

,FeaTyp,Cor,CorAbs,Log,LogBst,LogCor,LogCorAbs,Auc
anaemia,Cat,0.040565,0.040565,NaN,NaN,NaN,NaN,0.521244
diabetes,Cat,0.030201,0.030201,NaN,NaN,NaN,NaN,0.515691
high_blood_pressure,Cat,0.053208,0.053208,NaN,NaN,NaN,NaN,0.526604
sex,Cat,0.008830,0.008830,NaN,NaN,NaN,NaN,0.504428
smoking,Cat,-0.013503,0.013503,NaN,NaN,NaN,NaN,0.506525
age,Num,0.226820,0.226820,-,-5.0,0.215981,0.215981,0.616009
creatinine_phosphokinase,Num,0.085204,0.085204,-,-32.0,0.057817,0.057817,0.526293
ejection_fraction,Num,-0.252100,0.252100,+,19.0,-0.299468,0.299468,0.663780
platelets,Num,-0.069756,0.069756,+,26.0,-0.087695,0.087695,0.544275
serum_creatinine,Num,0.312228,0.312228,+,15.0,0.359451,0.359451,0.725183


# Decision Tree Classifier

In [12]:
dvt = DictVectorizer(sparse=False)
dicTra = dfrTra.to_dict(orient='records')
X_Tra = dvt.fit_transform(dicTra)

dicVal = dfrVal.to_dict(orient='records')
X_Val = dvt.transform(dicVal)

In [13]:
dpt = 3

# Train
dtc = DecisionTreeClassifier(max_depth=dpt)
dtc.fit(X_Tra, y_Tra)

# Predict on Train    
y_PrdTra = dtc.predict_proba(X_Tra)[:, 1]
floAucTra = roc_auc_score(y_Tra, y_PrdTra)

# Predict on Validation    
y_PrdVal = dtc.predict_proba(X_Val)[:, 1]
floAucVal = roc_auc_score(y_Val, y_PrdVal)
    
print( (dpt , floAucTra , floAucVal))
print(export_text(dtc, feature_names=dvt.get_feature_names_out().tolist()))

(3, 0.902824858757062, 0.8625146886016452)
|--- time <= 73.50
|   |--- age <= 66.50
|   |   |--- serum_sodium <= 134.50
|   |   |   |--- class: 1
|   |   |--- serum_sodium >  134.50
|   |   |   |--- class: 1
|   |--- age >  66.50
|   |   |--- ejection_fraction <= 47.50
|   |   |   |--- class: 1
|   |   |--- ejection_fraction >  47.50
|   |   |   |--- class: 1
|--- time >  73.50
|   |--- serum_creatinine <= 1.50
|   |   |--- serum_creatinine <= 0.65
|   |   |   |--- class: 1
|   |   |--- serum_creatinine >  0.65
|   |   |   |--- class: 0
|   |--- serum_creatinine >  1.50
|   |   |--- creatinine_phosphokinase <= 65.00
|   |   |   |--- class: 0
|   |   |--- creatinine_phosphokinase >  65.00
|   |   |   |--- class: 1



In [14]:
lstDpt = [1,2,3,4,5,6,8,10,20]
lstDpt

[1, 2, 3, 4, 5, 6, 8, 10, 20]

In [15]:
for dpt in lstDpt:
    # Train
    dtc = DecisionTreeClassifier(max_depth=dpt , random_state=1)
    dtc.fit(X_Tra, y_Tra)

    # Predict on Train    
    y_PrdTra = dtc.predict_proba(X_Tra)[:, 1]
    floAucTra = roc_auc_score(y_Tra, y_PrdTra)

    # Predict on Validation    
    y_PrdVal = dtc.predict_proba(X_Val)[:, 1]
    floAucVal = roc_auc_score(y_Val, y_PrdVal)

    print( (dpt , floAucTra.round(4) , floAucVal.round(4) ) )

(1, 0.7845, 0.8343)
(2, 0.8787, 0.8754)
(3, 0.9028, 0.8625)
(4, 0.9359, 0.8778)
(5, 0.9667, 0.8972)
(6, 0.9886, 0.8179)
(8, 0.9999, 0.8032)
(10, 1.0, 0.7914)
(20, 1.0, 0.7914)


<div class="alert alert-block alert-success">
<b>max_dept = 5 : </b>
    provide a good model!
</div>

In [16]:
lstDpt = [4,5,6]     # around 5
lstMsf = [2, 4, 8 , 10 , 12, 16 ]
lstDpt , lstMsf

([4, 5, 6], [2, 4, 8, 10, 12, 16])

In [17]:
# store the difference of Risk in a dataFrame to sort later
dfrAuc = pda.DataFrame(columns=['Dpt' , 'Msf' , 'AucTra' , 'AucVal'])

for dpt in lstDpt:
    for msf in lstMsf:
        # Train
        dtc = DecisionTreeClassifier(max_depth=dpt , min_samples_leaf=msf , random_state=1)
        dtc.fit(X_Tra, y_Tra)

        # Predict on Train    
        y_PrdTra = dtc.predict_proba(X_Tra)[:, 1]
        floAucTra = roc_auc_score(y_Tra, y_PrdTra)

        # Predict on Validation    
        y_PrdVal = dtc.predict_proba(X_Val)[:, 1]
        floAucVal = roc_auc_score(y_Val, y_PrdVal)

        #print( (dpt , msf , floAucTra.round(4) , floAucVal.round(4) ) )
        dfrAuc.loc[len(dfrAuc)] = [dpt , msf , floAucTra , floAucVal ]
# best 12
dfrAuc.sort_values(by='AucVal' , ascending=False)[:12]

,Dpt,Msf,AucTra,AucVal
3,4.0,10.0,0.927613,0.938308
14,6.0,8.0,0.940042,0.927732
2,4.0,8.0,0.934110,0.923032
8,5.0,8.0,0.938630,0.920682
9,5.0,10.0,0.933475,0.914219
15,6.0,10.0,0.933475,0.914219
1,4.0,4.0,0.937288,0.908343
0,4.0,2.0,0.947316,0.893067
11,5.0,16.0,0.919562,0.892479
5,4.0,16.0,0.919562,0.892479


<div class="alert alert-block alert-success">
<b>max_dept = 4 and min_samples_leaf = 10 : </b>
    provide the best model with AUC = <b>0.938308.
</div>

### Decision Tree Classifier: Final

In [18]:
# Decision Tree Classifier: Final¶
dtc = DecisionTreeClassifier(max_depth=4 , min_samples_leaf=10 , random_state=1)
dtc.fit(X_Tra, y_Tra)
# Predict on Validation    
y_PrdVal = dtc.predict_proba(X_Val)[:, 1]
floAucValDtc = roc_auc_score(y_Val, y_PrdVal)

floAucValDtc.round(5)

0.93831

### Decision Tree Classifier: Prediction on Test

In [19]:
# Predict on Validation    
dicTst = dfrTst.to_dict(orient='records')
X_Tst = dvt.transform(dicTst)

y_PrdTst = dtc.predict_proba(X_Tst)[:, 1]
floAucTstDct = roc_auc_score(y_Tst, y_PrdTst)
floAucTstDct

0.8734472049689441

# Random Forest

In [20]:
dvt = DictVectorizer(sparse=False)
dicTra = dfrTra.to_dict(orient='records')
X_Tra = dvt.fit_transform(dicTra)

dicVal = dfrVal.to_dict(orient='records')
X_Val = dvt.transform(dicVal)

In [21]:
# number of estimators 
lstEst = [ 20, 30, 40, 50, 60]
# max depth
lsrDpt = [5 ,10 ,15]
# Min sample leaf
lstMsf = [2 ,4 ,6 ,8 ,10 ,12 ]

In [22]:
# store the difference of Risk in a dataFrame to sort later
dfrAuc = pda.DataFrame(columns=['Est', 'Dpt' , 'Msf' , 'AucTra' , 'AucVal'])

for est in lstEst:
    for dpt in lstDpt:
        for msf in lstMsf:
            
            modRfo = RandomForestClassifier(n_estimators=est ,
                                    max_depth=dpt ,
                                    min_samples_leaf=msf ,
                                    random_state=1)
            modRfo.fit(X_Tra, y_Tra)
            
            # Predict on Train    
            y_PrdTra = modRfo.predict_proba(X_Tra)[:, 1]
            floAucTra = roc_auc_score(y_Tra, y_PrdTra)

            # Predict on Validation    
            y_PrdVal = modRfo.predict_proba(X_Val)[:, 1]
            floAucVal = roc_auc_score(y_Val, y_PrdVal)
            
            dfrAuc.loc[len(dfrAuc)] = [est , dpt , msf , floAucTra , floAucVal ]
# best 12
dfrAuc.sort_values(by='AucVal' , ascending=False)[:12]

,Est,Dpt,Msf,AucTra,AucVal
69,50.0,6.0,8.0,0.958616,0.933020
51,40.0,6.0,8.0,0.957345,0.930670
33,30.0,6.0,8.0,0.956215,0.930670
39,40.0,4.0,8.0,0.949576,0.929495
21,30.0,4.0,8.0,0.949859,0.928320
45,40.0,5.0,8.0,0.954661,0.927145
27,30.0,5.0,8.0,0.954520,0.925969
15,20.0,6.0,8.0,0.961158,0.924794
63,50.0,5.0,8.0,0.956638,0.923619
87,60.0,6.0,8.0,0.960311,0.922444


<div class="alert alert-block alert-success">
<b>estimators= 50 ; max_depth=6.0 , min_samples_leaf=8.0 : </b>
    provide the best model with AUC = <b>0.933020</b>.
</div>

### Random Forest: Final

In [23]:
modRfo = RandomForestClassifier(n_estimators=50 ,
                        max_depth=6 ,
                        min_samples_leaf=8 ,
                        random_state=1)
modRfo.fit(X_Tra, y_Tra)

# Predict on Validation    
y_PrdVal = modRfo.predict_proba(X_Val)[:, 1]
floAucValRfo = roc_auc_score(y_Val, y_PrdVal)
floAucValRfo.round(5)

0.93302

### Random Forest: Prediction on Test

In [24]:
# Predict on Test
dicTst = dfrTst.to_dict(orient='records')
X_Tst = dvt.transform(dicTst)

y_PrdTst = modRfo.predict_proba(X_Tst)[:, 1]
floAucTstRfo = roc_auc_score(y_Tst, y_PrdTst)
floAucTstRfo

0.9565217391304348

# xgboost

In [25]:
dvt = DictVectorizer(sparse=False)
dicTra = dfrTra.to_dict(orient='records')
X_Tra = dvt.fit_transform(dicTra)

dicVal = dfrVal.to_dict(orient='records')
X_Val = dvt.transform(dicVal)

In [26]:
lstFea = dvt.get_feature_names_out().tolist()
lstFea

['age',
 'anaemia=0',
 'anaemia=1',
 'creatinine_phosphokinase',
 'diabetes=0',
 'diabetes=1',
 'ejection_fraction',
 'high_blood_pressure=0',
 'high_blood_pressure=1',
 'platelets',
 'serum_creatinine',
 'serum_sodium',
 'sex=0',
 'sex=1',
 'smoking=0',
 'smoking=1',
 'time']

In [27]:
# dma for Xgboost
dmaTra = xgb.DMatrix(X_Tra, label=y_Tra, feature_names=lstFea)
dmaVal = xgb.DMatrix(X_Val, label=y_Val, feature_names=lstFea)

In [28]:
watchlist = [(dmaTra, 'train'), (dmaVal, 'val')]

In [29]:
xgb_params = {
    'eta': 0.3, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}

modXgb = xgb.train(xgb_params, 
                   dmaTra, 
                   num_boost_round=100 ,
                   evals=watchlist ,
                   verbose_eval=20
                  )

[0]	train-auc:0.95184	val-auc:0.84959
[20]	train-auc:0.99986	val-auc:0.93302
[40]	train-auc:1.00000	val-auc:0.93655
[60]	train-auc:1.00000	val-auc:0.93420
[80]	train-auc:1.00000	val-auc:0.93302
[99]	train-auc:1.00000	val-auc:0.93067


In [30]:
# Predict on Train    
y_PrdTra = modXgb.predict(dmaTra)
floAucTra = roc_auc_score(y_Tra, y_PrdTra)

# Predict on Validation   
y_PrdVal = modXgb.predict(dmaVal)
floAucVal = roc_auc_score(y_Val, y_PrdVal)

floAucTra , floAucVal

(1.0, 0.9306698002350177)

In [31]:
# Eta
lstEta = [ 0.1 , 0.3 , 0.5]
# max depth
lstDpt = [2, 4, 6, 8 , 16]
# Min sample leaf
lstMsf = [ 2 ,4 ,6 ]
# num_boost_round
lstNbr = [75, 100, 125 , 150, 200]

In [37]:
# store the difference of Risk in a dataFrame to sort later
dfrAuc = pda.DataFrame(columns=['Eta', 'Dpt' , 'Msf' , 'Nbr' , 'AucTra' , 'AucVal'])

for eta in lstEta:
    for dpt in lstDpt:
        for msf in lstMsf:
            for nbr in lstNbr:
                
                xgb_params = { 
                    'eta': eta, 
                    'max_depth': dpt,
                    'min_child_weight': msf,
                    'objective': 'binary:logistic',
                    'eval_metric': 'auc',
                    'nthread': 8, 'seed': 1, 'verbosity': 1  }

                modXgb = xgb.train(xgb_params, dmaTra, num_boost_round=nbr ,
                               evals=watchlist , verbose_eval=nbr*2 )

                # Predict on Train    
                y_PrdTra = modXgb.predict(dmaTra)
                floAucTra = roc_auc_score(y_Tra, y_PrdTra)

                # Predict on Validation   
                y_PrdVal = modXgb.predict(dmaVal)
                floAucVal = roc_auc_score(y_Val, y_PrdVal)

                print((eta , dpt , msf , nbr ,floAucTra , floAucVal))

                dfrAuc.loc[len(dfrAuc)] = [eta , dpt , msf , nbr, floAucTra , floAucVal ]

[0]	train-auc:0.87853	val-auc:0.87192
[74]	train-auc:0.98023	val-auc:0.92362
(0.1, 2, 2, 75, 0.9802259887005649, 0.9236192714453584)
[0]	train-auc:0.87853	val-auc:0.87192
[99]	train-auc:0.98573	val-auc:0.91892
(0.1, 2, 2, 100, 0.9857344632768361, 0.9189189189189189)
[0]	train-auc:0.87853	val-auc:0.87192
[124]	train-auc:0.98983	val-auc:0.92244
(0.1, 2, 2, 125, 0.9898305084745763, 0.9224441833137484)
[0]	train-auc:0.87853	val-auc:0.87192
[149]	train-auc:0.99251	val-auc:0.91539
(0.1, 2, 2, 150, 0.9925141242937853, 0.9153936545240893)
[0]	train-auc:0.87853	val-auc:0.87192
[199]	train-auc:0.99703	val-auc:0.92714
(0.1, 2, 2, 200, 0.9970338983050847, 0.927144535840188)
[0]	train-auc:0.87853	val-auc:0.87192
[74]	train-auc:0.96624	val-auc:0.92127
(0.1, 2, 4, 75, 0.9662429378531073, 0.9212690951821386)
[0]	train-auc:0.87853	val-auc:0.87192
[99]	train-auc:0.97218	val-auc:0.92362
(0.1, 2, 4, 100, 0.9721751412429379, 0.9236192714453584)
[0]	train-auc:0.87853	val-auc:0.87192
[124]	train-auc:0.97599	

(0.1, 16, 2, 100, 0.9992937853107345, 0.9259694477085781)
[0]	train-auc:0.90410	val-auc:0.89835
[124]	train-auc:0.99958	val-auc:0.92714
(0.1, 16, 2, 125, 0.9995762711864407, 0.9271445358401881)
[0]	train-auc:0.90410	val-auc:0.89835
[149]	train-auc:0.99972	val-auc:0.92832
(0.1, 16, 2, 150, 0.9997175141242938, 0.9283196239717979)
[0]	train-auc:0.90410	val-auc:0.89835
[199]	train-auc:1.00000	val-auc:0.92714
(0.1, 16, 2, 200, 1.0, 0.927144535840188)
[0]	train-auc:0.87853	val-auc:0.87192
[74]	train-auc:0.97797	val-auc:0.92832
(0.1, 16, 4, 75, 0.9779661016949153, 0.9283196239717979)
[0]	train-auc:0.87853	val-auc:0.87192
[99]	train-auc:0.98164	val-auc:0.92832
(0.1, 16, 4, 100, 0.981638418079096, 0.9283196239717978)
[0]	train-auc:0.87853	val-auc:0.87192
[124]	train-auc:0.98517	val-auc:0.93302
(0.1, 16, 4, 125, 0.9851694915254238, 0.9330199764982374)
[0]	train-auc:0.87853	val-auc:0.87192
[149]	train-auc:0.98743	val-auc:0.93184
(0.1, 16, 4, 150, 0.9874293785310735, 0.9318448883666275)
[0]	train-

(0.3, 8, 2, 200, 1.0, 0.9224441833137486)
[0]	train-auc:0.87853	val-auc:0.87192
[74]	train-auc:0.99251	val-auc:0.93420
(0.3, 8, 4, 75, 0.9925141242937854, 0.9341950646298474)
[0]	train-auc:0.87853	val-auc:0.87192
[99]	train-auc:0.99605	val-auc:0.92714
(0.3, 8, 4, 100, 0.9960451977401129, 0.927144535840188)
[0]	train-auc:0.87853	val-auc:0.87192
[124]	train-auc:0.99760	val-auc:0.92479
(0.3, 8, 4, 125, 0.9975988700564972, 0.9247943595769683)
[0]	train-auc:0.87853	val-auc:0.87192
[149]	train-auc:0.99760	val-auc:0.92597
(0.3, 8, 4, 150, 0.9975988700564972, 0.9259694477085781)
[0]	train-auc:0.87853	val-auc:0.87192
[199]	train-auc:0.99887	val-auc:0.91657
(0.3, 8, 4, 200, 0.9988700564971752, 0.9165687426556992)
[0]	train-auc:0.86864	val-auc:0.87720
[74]	train-auc:0.96808	val-auc:0.94242
(0.3, 8, 6, 75, 0.9680790960451978, 0.9424206815511165)
[0]	train-auc:0.86864	val-auc:0.87720
[99]	train-auc:0.97034	val-auc:0.94360
(0.3, 8, 6, 100, 0.9703389830508475, 0.9435957696827262)
[0]	train-auc:0.8686

[149]	train-auc:0.99887	val-auc:0.91539
(0.5, 6, 4, 150, 0.9988700564971752, 0.9153936545240893)
[0]	train-auc:0.87853	val-auc:0.87192
[199]	train-auc:0.99944	val-auc:0.91774
(0.5, 6, 4, 200, 0.9994350282485875, 0.917743830787309)
[0]	train-auc:0.86864	val-auc:0.87720
[74]	train-auc:0.97034	val-auc:0.92597
(0.5, 6, 6, 75, 0.9703389830508475, 0.9259694477085781)
[0]	train-auc:0.86864	val-auc:0.87720
[99]	train-auc:0.97189	val-auc:0.92832
(0.5, 6, 6, 100, 0.9718926553672317, 0.9283196239717979)
[0]	train-auc:0.86864	val-auc:0.87720
[124]	train-auc:0.97274	val-auc:0.92714
(0.5, 6, 6, 125, 0.9727401129943501, 0.9271445358401881)
[0]	train-auc:0.86864	val-auc:0.87720
[149]	train-auc:0.97401	val-auc:0.92479
(0.5, 6, 6, 150, 0.9740112994350282, 0.9247943595769683)
[0]	train-auc:0.86864	val-auc:0.87720
[199]	train-auc:0.97698	val-auc:0.91774
(0.5, 6, 6, 200, 0.9769774011299436, 0.9177438307873091)
[0]	train-auc:0.90410	val-auc:0.89835
[74]	train-auc:1.00000	val-auc:0.93067
(0.5, 8, 2, 75, 1.0,

In [32]:
# best
dfrAuc.sort_values(by='AucVal' , ascending=False)[:12]

,Est,Dpt,Msf,AucTra,AucVal
69,50.0,6.0,8.0,0.958616,0.933020
51,40.0,6.0,8.0,0.957345,0.930670
33,30.0,6.0,8.0,0.956215,0.930670
39,40.0,4.0,8.0,0.949576,0.929495
21,30.0,4.0,8.0,0.949859,0.928320
45,40.0,5.0,8.0,0.954661,0.927145
27,30.0,5.0,8.0,0.954520,0.925969
15,20.0,6.0,8.0,0.961158,0.924794
63,50.0,5.0,8.0,0.956638,0.923619
87,60.0,6.0,8.0,0.960311,0.922444


<div class="alert alert-block alert-success">
<b>eta= 0.5 ; max_depth=2 , min_samples_leaf=2 , num_boost_round=125 : </b>
    provide the best model with AUC = <b>0.945946</b>.
</div>

### xgboost: Final

In [33]:
xgb_params = { 
    'eta': 0.5, 
    'max_depth': 2,
    'min_child_weight': 2,
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'nthread': 8, 'seed': 1, 'verbosity': 1  }

modXgb = xgb.train(xgb_params, dmaTra, num_boost_round=125 ,
               evals=watchlist , verbose_eval=125 )

# Predict on Validation   
y_PrdVal = modXgb.predict(dmaVal)
floAucValXgb = roc_auc_score(y_Val, y_PrdVal)
floAucValXgb.round(5)

[0]	train-auc:0.87853	val-auc:0.87192
[124]	train-auc:1.00000	val-auc:0.94595


0.94595

### xgboost: Prediction on Test

In [34]:
dicTst = dfrTst.to_dict(orient='records')
X_Tst = dvt.transform(dicTst)
dmaTst = xgb.DMatrix(X_Tst, label=y_Tst, feature_names=lstFea)
# Predict on TST   
y_PrdTst = modXgb.predict(dmaTst)
floAucTstXgb = roc_auc_score(y_Tst, y_PrdTst)
floAucTstXgb

0.9254658385093167